<a href="https://colab.research.google.com/github/Maicken052/MACC/blob/main/Modelado_Molecular_Para_la_Optimizaci%C3%B3n_de_Compuestos_y_Caracterizaci%C3%B3n_de_Propiedades_Mediante_Grafos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Modelado Molecular Para la Optimización de Compuestos y Caracterización de Propiedades Mediante Grafos**

##**Paquetes para instalación e importación**

Se usó la librería igraph para la modelación de los grafos y mathplotlib junto a itertools como librerías auxiliares.


In [ ]:
pip install igraph;

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 11.7 MB/s eta 0:00:00


In [ ]:
pip install cairocffi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 1.7 MB/s eta 0:00:00


In [ ]:
pip install numpy

In [ ]:
import igraph as ig
import matplotlib.pyplot as plt
import numpy as np
import itertools

##**Moléculas propuestas**

Ejemplos de moléculas resueltas por el algoritmo:
*   **CH3COOH** (Ácido acético)
*   **CO2** (Dióxido de carbono)
*   **CH3OH** (Metanol)
*   **C2H4** (Etileno)
*   **CH2O** (Formaldehído)
*   **HCOOH** (Ácido fórmico)
*   **CH3NH2** (Metilamina)
*   **NH3** (Amoníaco)
*   **H2O** (Oxidano o Agua)
*   **HCONH2** (Formamida)
*   **CH3C3H5** (Metilciclopropano)
*   **CH3CH2CH2CH2CH3** (Pentano)

Observaciones:
*   Note que la última fórmula está escrita en forma semidesarrollada, esto para que el algoritmo no tenga que hacer tantas ejecuciones, que es lo que ocurre cuando ponemos la fórmula molecular resumida (C5H12).
*   En caso de que la molécula tenga isomeros estructurales, el algoritmo usará el primero que logre resolver, por lo que no es preciso en este aspecto. Es posible que escribiendo la fórmula en forma semidesarrollada en cierto orden (el conveniente para el usuario), de como resultado el isomero deseado.
*   Por la manera en que trabaja el algoritmo, no es posible que cree un ciclo aromático.
*   Después de los 13 átomos, el algoritmo puede tomar tiempos muy extensos (de 7 minutos a más de una hora dependiendo del tamaño) para ejecutarse dependiendo del tipo de molécula que se le pida, dado que debe probar todas las posibles matrices de adyacencia hasta que una de ellas cumpla con los requisitos.
*   Si queremos crear un alcano, debemos pedir solo enlaces simples, ya que esta es su característica principal.












##**Ingresar fórmula molecular**
En este apartado el programa pide al ususario la fórmula y crea una lista que descompone el número de átomos dentro de ella para que cada uno funcione como un vértice independiente.

In [ ]:
atomos_permitidos  = ['C','H','O','N']
letras = [chr(i) for i in range(65, 91)] # 65 a 90 son los códigos ASCII para A a Z
no_atomos = [letra for letra in letras if letra not in atomos_permitidos] #Letras que no corresponden a los átomos permitidos

def lista_vertices(formula):
  atomos = [] #lista de vértices compuesta por los átomos de la fórmula
  repeticiones = [] #Número de veces que se repite cada átomo según la forma en la que se escribió la fórmula. ej: CH3CH3 retorna repeticiones [1, 3, 1, 3]
  dos_digitos = False #Si se encontró un número de dos dígitos previamente, pasa a la siguiente iteración

  for i in range(len(formula)):
    if dos_digitos:
      dos_digitos = False
      continue
    else:
      if formula[i] in atomos_permitidos:#Verifica si es uno de los átomos permitidos
        atomos.append(formula[i]) #Lo añade a la lista de vertices
        try:
          if(int(formula[i+1])>0): #Verifica si hay más de una molécula,
            continue
        except:
          repeticiones.append(1) #Si no hay más de una molécula, añade un 1 a la lista de números
          continue

      elif formula[i] == ' ' and i == len(formula) - 1: #Si se cuentra un espacio vacio al final, se termina el proceso
        break

      elif formula[i] in no_atomos or formula[i] == ' ': #En caso de no serlo o de encontrar un espacio vacio en medio de la formula, muestra un error
        return "Fórmula molecular inválida, por favor revise lo que ingresó",0

      elif int(formula[i]) > 0: #Verifica si es un número
        if i < len(formula) - 1: #Y no es el último número de la cadena
          if formula[i+1] in atomos_permitidos or formula[i+1] in no_atomos or formula[i+1] == ' ':
            repeticiones.append(int(formula[i])) #Si lo que le sigue es una letra, añade el número a repeticiones

          elif(int(formula[i+1]) >= 0):
            repeticiones.append(int(formula[i]+formula[i+1])) #Si le sigue un número, significa que era un número de 2 dígitos, por lo que se concatenan y se activa la flag
            dos_digitos = True
        else: #Si es el ultimo de la cadena
          repeticiones.append(int(formula[i])) #Añade el número

  return atomos, repeticiones

formula = input("Ingrese la fórmula molecular: ")
enlace = input("Ingrese el número de enlaces máximo deseado (simple, doble, triple) (Si no tiene esta información, ingrese 3): ")
formula = formula.upper() #Se coloca el mayúsculas
atomos = lista_vertices(formula)[0]
repeticiones = lista_vertices(formula)[1]
vertices = []

if repeticiones != 0: #Si no ocurrió un error
  #Se agrega el átomo correspondiente a los vértices según su número de repeticiones
  for i in range(len(atomos)):
      for j in range(repeticiones[i]):
          vertices.append(atomos[i])
  #print(f'Vértices: {vertices}')
  #print(f'Número máximo de enlaces posibles: {enlace}')
else:
  print(atomos)

##**Creación de la matriz de adyacencia**
En este apartado se crea la matriz de adyacencia de la fórmula dada según las reglas de la química orgánica.

### *Funciones necesarias*

In [ ]:
'''Cualquiera de estas funciones nos dice el estado de un átomo en un momento dado.
0 si necesita enlaces, 1 si los enlaces están completos, 2 en cualquier otro caso'''

def O_completo(matriz, i, vertice): #El oxígeno necesita 2 enlaces para estar completo
  suma_fila = sum(matriz[i])
  if (vertice == "O" and suma_fila < 2):
    ret = 0
  elif (vertice == "O" and suma_fila == 2):
    ret = 1
  else:
    ret = 2
  return ret

def H_completo(matriz, i, vertice): #El hidrógeno necesita 1 enlaces para estar completo
  suma_fila = sum(matriz[i])
  if (vertice == "H" and suma_fila < 1):
    ret=0
  elif (vertice == "H" and suma_fila == 1):
    ret = 1
  else:
    ret = 2
  return ret

def N_completo(matriz, i, vertice): #El Nitrógeno necesita 3 enlaces para estar completo
  suma_fila = sum(matriz[i])
  if (vertice == "N" and suma_fila < 3):
    ret=0
  elif (vertice == "N" and suma_fila == 3):
    ret = 1
  else:
    ret = 2
  return ret

def C_completo(matriz, i, vertice): #El carbono necesita 4 enlaces para estar completo
  suma_fila = sum(matriz[i])
  if (vertice == "C" and suma_fila < 4):
    ret = 0
  elif (vertice == "C" and suma_fila == 4):
    ret = 1
  else:
    ret = 2
  return ret

# Verifica si el grafo es conexo
def matriz_conexa(matriz):
  visitado = [False] * len(matriz)
  stack = [0]
  while stack:
    vertice = stack.pop()
    visitado[vertice] = True
    for vecino, es_adyacente in enumerate(matriz[vertice]):
      if es_adyacente and not visitado[vecino]:
        stack.append(vecino)
  return all(visitado)

#Crea una matriz de 0's
def crear_matriz(vertices):
  tamano = len(vertices)
  matriz = [[0] * tamano for i in range(tamano)]
  return matriz

#Verificar las sumas en las filas de la matriz de adyacencia (que todos los átomos tengan sus enlaces completos)
def enlaces_completos(matriz, vertices):
  for i in range(len(vertices)):
    vertice = vertices[i]
    suma_fila = sum(matriz[i])
    if (vertice == "O" and suma_fila != 2) or (vertice == "H" and suma_fila != 1) or (vertice == "N" and suma_fila != 3 ) or (vertice == "C" and suma_fila != 4):
      return False
  return True

#Recibe una lista ordenada de vértices y retorna la matriz y la lista en caso de que cumpla las condiciones, de lo contrario retorna una tupla de 0's
def llenar_matriz(matriz, vertices, enlaces):
  for row in range(len(matriz)):
    for col in range(len(matriz)):
      for i in range(enlaces): #Realiza el proceso 3 veces (caso de tener que sumar los 3 enlaces del triple enlace de carbono)
        if(row == col): #No existen los bucles en estos grafos
          break
        if(matriz[row][col] <= 3): #Si aún no ha llegado al máximo de enlaces

          if(vertices[row] == "O" and vertices[col] == "O") or (vertices[row ]== 'H' and vertices[col] == 'H'): #Si los átomos son iguales (en el caso del oxígeno y del hidrógeno), no los une
            break

          elif((vertices[row] == "H" or vertices[col] == "H") and matriz[row][col] == 1): #Si uno de los átomos es un hidrógeno y ya tiene su enlace, pasa al siguiente índice
            break

          else:
            matriz[row][col] += 1 #Intenta añadir un enlace entre ambos
            matriz[col][row] += 1 #Esto por simetría

            #Si el átomo de la fila está completo / no tiene forma de obtener más enlaces
              #Revisa si el átomo de la columna está completo / no tiene forma de obtener más enlaces
              #Si ambas se cumplen, se sigue normal
              #Si la que está en las columnas no se cumple (Tiene más enlaces de los que debería), deshace el enlace
            #Si en vez de eso el átomo de la fila no podía conseguir más enlaces, deshace el enlace

            if O_completo(matriz, row ,vertices[row]) <= 1:
              if H_completo(matriz, col, vertices[col]) <= 1:
                continue
              elif O_completo(matriz, col, vertices[col]) <= 1:
                continue
              elif C_completo(matriz, col, vertices[col]) <= 1:
                continue
              elif N_completo(matriz, col, vertices[col]) <= 1:
                continue
              else:
                matriz[row][col] -= 1
                matriz[col][row] -= 1
                break

            elif H_completo(matriz, row, vertices[row]) <= 1:
              if H_completo(matriz, col, vertices[col])<=1:
                continue
              elif O_completo(matriz, col, vertices[col]) <= 1:
                continue
              elif C_completo(matriz, col, vertices[col]) <= 1:
                continue
              elif N_completo(matriz, col, vertices[col]) <= 1:
                continue
              else:
                matriz[row][col] -= 1
                matriz[col][row] -= 1
                break

            elif C_completo(matriz, row, vertices[row]) <= 1:
              if H_completo(matriz, col, vertices[col]) <= 1:
                continue
              elif O_completo(matriz, col, vertices[col]) <= 1:
                continue
              elif C_completo(matriz, col, vertices[col]) <= 1:
                continue
              elif N_completo(matriz, col, vertices[col]) <= 1:
                continue
              else:
                matriz[row][col] -= 1
                matriz[col][row] -= 1
                break

            elif N_completo(matriz, row, vertices[row]) <= 1:
              if H_completo(matriz, col, vertices[col]) <= 1:
                continue
              elif O_completo(matriz, col, vertices[col]) <= 1:
                continue
              elif C_completo(matriz, col, vertices[col]) <= 1:
                continue
              elif N_completo(matriz, col, vertices[col]) <= 1:
                continue
              else:
                matriz[row][col] -= 1
                matriz[col][row] -= 1
                break
            #Si ninguno acepta
            else:
                matriz[row][col] -= 1
                matriz[col][row] -= 1
                break
        else:
          break
  if(matriz_conexa(matriz) and enlaces_completos(matriz, vertices)): #Verifica si la matriz es conexa y si están todos los átomos con su respectivo número de enlaces completos
    return matriz, vertices
  return 0,0

#Probamos todas las permutaciones de listas de vértices posibles
def formula_a_matriz(matriz, vertices, enlaces):
  res = llenar_matriz(matriz, vertices, enlaces)
  if res[0] != 0: #Si no retornó un 0
    return res
  else:
    #Hace las posibles permutaciones y elimina las repeticiones (Como cada átomo es considerado distinto, podemos tener resultados en los que el orden sea el mismo, aunque el programa los considere elementos distintos)
    perm = set(itertools.permutations(vertices))
    perm = [list(permutacion) for permutacion in perm]
    tamano = len(vertices)
    for p in perm:
      matriz = [[0] * tamano for _ in range(tamano)] #reinicia la matriz de los resultados anteriores
      res = llenar_matriz(matriz, p, enlaces)
      if res[0] != 0:
        return res
  return 0,0

### *Resultado obtenido*

In [ ]:
# Creamos la matriz y clonamos los vértices
vertices_ = []
for i in vertices:
  vertices_.append(i)
matriz = crear_matriz(vertices_)
if(int(enlace)<0):
    sol = formula_a_matriz(matriz, vertices_, 3) #Corre el solucionador
else:
  sol = formula_a_matriz(matriz, vertices_, int(enlace)) #Corre el solucionador
matriz = sol[0]
vertices = sol[1]
if(matriz == 0):#Si no nos da un matriz conexa con enlaces completos
  print('Molécula invalida')
else:
  print('Matriz de adyacencia:')
  for fila in matriz: #Imprime la matriz solucionada
    print(" ".join(map(str, fila)))


In [ ]:
print(vertices)

##**Estructura de Lewis (Grafo)**
En este apartado podemos ver el resultado de la matriz de adyacencia, que nos muestra cada enlace y átomo, con su color correspondiente (según la normativa internacional).

In [ ]:
def visualizar_grafo(matriz, formula, vertices):
  grafo = ig.Graph.Adjacency(matriz, mode="undirected")
  fig, ax = plt.subplots(figsize=(len(matriz),len(matriz)))
  ax.set_title(formula, fontsize=25, fontweight='bold')  #Título de la fórmula

  #Color correspondiente a cada átomo
  colors = []
  for i in vertices:
    if i == 'C':
      colors.append('dimgrey')
    elif i == 'H':
      colors.append('white')
    elif i == 'N':
      colors.append('royalblue')
    else:
      colors.append('lightcoral')
  grafo.vs['color'] = colors

  # Asignar nombres a los vértices
  grafo.vs["name"] = vertices

  ig.plot(
      grafo,
      target=ax,
      vertex_size=50,
      vertex_label=vertices,
      vertex_frame_color="black",
      vertex_frame_width=2.0,
      vertex_label_size=30.0,
      edge_width=4,
  )

  plt.show()
  return grafo

In [ ]:
grafo = visualizar_grafo(matriz, formula, vertices)

##A continuación, se procede a caracterizar la molécula obtenida

##**Masa de la molécula**

###*Masa de cada átomo*

In [ ]:
# Asignar masas a los vértices
masas = []
for i in vertices:
  if i == "C":
    masas.append(12.011)
  elif i == "H":
    masas.append(1.00784)
  elif i == "O":
    masas.append(15.999)
  elif i == "N":
    masas.append(14.0067)
grafo.vs["masa"] = masas

###*Masa total*

In [ ]:
def recorrer_bfs_por_masa(grafo, inicio=0):
    """
    Realiza un recorrido por búsqueda en anchura en el grafo e imprime la masa de cada vértice.

    Args:
    - grafo: Grafo de igraph.
    - inicio: Índice del vértice de inicio para la búsqueda en anchura (por defecto, 0).

    Returns:
    - masa_total: Masa total de los vértices visitados.
    """
    cola = [inicio]
    visitados = set()
    masa_total = 0

    while cola:
      actual = cola.pop(0)
      if actual not in visitados:
        visitados.add(actual)
        vertice = grafo.vs[actual]
        masa = vertice["masa"]
        masa_total += masa
        nombre_vertice = vertice["name"]
        vecinos = grafo.neighbors(actual)
        cola.extend(vecinos)

    return masa_total

In [ ]:
# Llamar a la función de búsqueda en anchura con el grafo de ejemplo
masa_total_resultante = recorrer_bfs_por_masa(grafo)
print(f"Masa total de los vértices visitados: {masa_total_resultante} U")

###*Zona de masa máxima*

In [ ]:
def suma_masas_adyacentes(grafo):
    suma_maxima = 0
    vertice_max_suma = None

    for vertice_actual in grafo.vs:
        indice_actual = vertice_actual.index
        masa_actual = vertice_actual["masa"]
        vecinos = set(grafo.neighbors(indice_actual))
        suma_adyacentes = masa_actual + sum(grafo.vs[vecino]["masa"] for vecino in vecinos)

        if suma_adyacentes > suma_maxima:
            suma_maxima = suma_adyacentes
            vertice_max_suma = indice_actual

    return vertice_max_suma, suma_maxima

In [ ]:
# Llamar a la función para calcular la suma de masas y encontrar el vértice donde se maximiza
vertice_max_suma_masa, suma_maxima_masa = suma_masas_adyacentes(grafo)
print(f"El vértice donde se maximiza la suma de masas es: {grafo.vs[vertice_max_suma_masa]['name']}")
print(f"La suma máxima de masas es: {suma_maxima_masa} U")

##**Electronegatividad de la molécula**

### *Carga de cada átomo*

In [ ]:
# Asignar cargas a los vértices
cargas = []
for i in vertices:
  if i == "C":
    cargas.append(2.55)
  elif i == "H":
    cargas.append(2.2)
  elif i == "O":
    cargas.append(3.44)
  elif i == "N":
    cargas.append(3.04)
grafo.vs["electronegatividad"] = cargas

###*Electronegatividad total*

In [ ]:
def recorrer_bfs_por_carga(grafo, inicio=0):
    """
    Realiza un recorrido por búsqueda en anchura en el grafo e imprime la carga de cada vértice.

    Args:
    - grafo: Grafo de igraph.
    - inicio: Índice del vértice de inicio para la búsqueda en anchura (por defecto, 0).

    Returns:
    - carga_total: carga total de los vértices visitados.
    """
    cola = [inicio]
    visitados = set()
    carga_total = 0

    while cola:
        actual = cola.pop(0)
        if actual not in visitados:
            visitados.add(actual)
            vertice = grafo.vs[actual]
            carga = vertice["electronegatividad"]
            carga_total += carga
            vecinos = grafo.neighbors(actual)
            cola.extend(vecinos)

    return carga_total

In [ ]:
# Llamar a la función de búsqueda en anchura con el grafo de ejemplo
electronegatividad_total_resultante = recorrer_bfs_por_carga(grafo)
print(f"Carga total de los vértices visitados: {electronegatividad_total_resultante} kJ/mol")

###*Zona de electronegatividad máxima*

In [ ]:
def suma_cargas_adyacentes(grafo):
    suma_maxima = 0
    vertice_max_suma = None

    for vertice_actual in grafo.vs:
        indice_actual = vertice_actual.index
        carga_actual = vertice_actual["electronegatividad"]
        vecinos = set(grafo.neighbors(indice_actual))
        suma_adyacentes = carga_actual + sum(grafo.vs[vecino]["electronegatividad"] for vecino in vecinos)

        if suma_adyacentes > suma_maxima:
            suma_maxima = suma_adyacentes
            vertice_max_suma = indice_actual

    return vertice_max_suma, suma_maxima

In [ ]:
# Llamar a la función para calcular la suma de cargas y encontrar el vértice donde se maximiza
vertice_max_suma_carga, suma_maxima_carga = suma_cargas_adyacentes(grafo)
print(f"El vértice donde se maximiza la suma de electronegatividades es: {grafo.vs[vertice_max_suma_carga]['name']}")
print(f"La suma máxima de electronegatividad es: {suma_maxima_carga} kJ/mol")

##**Grupos funcionales presentes en la molécula**

### *Alcanos*

In [ ]:
def es_alcano(matriz, vertices):
  x = 0
  for i in range(len(vertices)):
    for j in range(len(vertices)):
      if ((matriz[i][j] == 2 or matriz[i][j] == 3) and vertices[i] == "C" and vertices[j] == "C") or (vertices[i] != "C" and vertices[i] != "H") or (vertices[j] != "C"and vertices[j] != "H"):
        return False
  return True

In [ ]:
alcano = es_alcano(matriz, vertices)
if alcano:
  print('Tiene como grupo funcional un alcano')
else:
  print('No tiene como grupo funcional un alcano')

### *Alquenos*

In [ ]:
def es_alqueno(matriz, vertices):
  x=0
  for i in range(len(vertices)):
    for j in range(len(vertices)):
      if ((matriz[i][j] == 3) and vertices[i] == "C" and vertices[j] == "C") or (vertices[i]!="C" and vertices[i]!="H") or (vertices[j]!="C"and vertices[j]!="H"):
        return False
      if(matriz[i][j]==2 and vertices[i] == "C" and vertices[j] == "C"):
        x=1

  return True if x==1 else False

In [ ]:
alqueno = es_alqueno(matriz, vertices)
if alqueno:
  print('Tiene como grupo funcional un alqueno')
else:
  print('No tiene como grupo funcional un alqueno')

### *Alquinos*

In [ ]:
def es_alquino(matriz, vertices):
  x = 0
  for i in range(len(vertices)):
    for j in range(len(vertices)):
      if (vertices[i] != "C" and vertices[i] != "H") or (vertices[j] != "C" and vertices[j] != "H"):
        return False
      if(matriz[i][j]==3 and vertices[i] == "C" and vertices[j] == "C"):
        x = 1
  return True if x == 1 else False

In [ ]:
alquino = es_alquino(matriz, vertices)
if alquino:
  print('Tiene como grupo funcional un alquino')
else:
  print('No tiene como grupo funcional un alquino')

###*Acido carboxílico*

In [ ]:
def es_acido_carboxilico(matriz, vertices):
  for i in range(len(vertices)):
    for j in range(len(vertices)):
      if matriz[i][j] == 2 and vertices[i] == "C" and vertices[j] == "O": #Revisa si existe un enlace doble entre un carbono y un oxígeno
        for h in range(len(vertices)):
          if matriz[i][h] == 1 and vertices[i] == "C" and vertices[h] == "O": #Revisa si existe un enlace doble entre ese carbono y otro oxígeno
            for k in range(len(vertices)):
              if matriz[h][k] == 1 and vertices[h] == "O" and vertices[k] == "H": #Revisa si existe un enlace doble entre ese oxígeno y un hidrógeno
                return True
  return False

In [ ]:
acido_carboxilico = es_acido_carboxilico(matriz,vertices)
if acido_carboxilico:
  print('Tiene como grupo funcional un ácido carboxílico')
else:
  print('No tiene como grupo funcional un ácido carboxílico')

###*Carbonilo*

In [ ]:
def es_carbonilo(matriz, vertices):
  for i in range(len(vertices)):
    for j in range(len(vertices)):
      if matriz[i][j] == 2 and vertices[i] == "C" and vertices[j] == "O" and not es_acido_carboxilico(matriz,vertices) and not es_ester(matriz,vertices): #Si existe un doble enlace entre un carbono y un oxígeno, y no tiene un ácido carboxilico como grupo principal
        return True
  return False

In [ ]:
carbonilo = es_carbonilo(matriz, vertices)
if carbonilo:
  print('Tiene como grupo funcional un carbonilo')
else:
  print('No tiene como grupo funcional un un carbonilo')

### Esteres

In [ ]:
def es_ester(matriz, vertices):
  for i in range(len(vertices)):
    for j in range(len(vertices)):
      if matriz[i][j] == 2 and vertices[i] == "C" and vertices[j] == "O":
        for h in range(len(vertices)):
          if matriz[i][h] == 1 and vertices[i] == "C" and vertices[h] == "O":
            for k in range(len(vertices)):
              if matriz[h][k] == 1 and vertices[k] == "C" and i != k:
                suma = 0
                for u in range(len(vertices)):
                  if matriz[k][u] == 1 and vertices[u] == "H":
                    suma += 1
                if(suma == 3):
                  return True
  return False

In [ ]:
ester = es_ester(matriz, vertices)
if ester:
  print('Tiene como grupo funcional un ester')
else:
  print('No tiene como grupo funcional un ester')

###*Amida*

In [ ]:
def es_amida(matriz, vertices):
  for i in range(len(vertices)):
    for j in range(len(vertices)):
      if matriz[i][j] == 2 and vertices[i] == "C" and vertices[j] == "O": #Si existe un doble enlace entre un carbono y un oxígeno
        for h in range(len(vertices)):
          if matriz[i][h] == 1 and vertices[i] == "C" and vertices[h] == "N": #Si ese carbono tiene un enlace con un nitrógeno
            return True
  return False

In [ ]:
amida = es_amida(matriz, vertices)
if amida:
  print('Tiene como grupo funcional una amida')
else:
  print('No tiene como grupo funcional una amida')

###*Amina primaria*

In [ ]:
def es_amina_primaria(matriz, vertices):
  for i in range(len(vertices)):
    for j in range(len(vertices)):
      if matriz[i][j] == 1 and vertices[i] == "N" and vertices[j] == "H": #Si existe un enlace entre un nitrógeno y un hidrógeno
        for h in range(len(vertices)):
          if matriz[i][h] == 1 and vertices[i] == "N" and vertices[h] == "H" and h != j and not es_amida(matriz, vertices): #Si existe un enlace con otro hidrógeno y no es una amida
            return True
  return False

In [ ]:
amina_p = es_amina_primaria(matriz, vertices)
if amina_p:
  print('Tiene como grupo funcional una amina primaria')
else:
  print('No tiene como grupo funcional una amina primaria')

###*Amina Secundaria*

In [ ]:
def es_amina_secundaria(matriz, vertices):
  for i in range(len(vertices)):
    for j in range(len(vertices)):
      if matriz[i][j] == 1 and vertices[i] == "N" and vertices[j] == "H": #Si existe un enlace entre un nitrógeno y un hidrógeno
        for h in range(len(vertices)):
          if not es_amina_primaria(matriz,vertices) and not es_amida(matriz, vertices):  #Si no existe un enlace con otro hidrógeno y no es una amida
            return True
  return False

In [ ]:
amina_s = es_amina_secundaria(matriz, vertices)
if amina_s:
  print('Tiene como grupo funcional una amina secundaria')
else:
  print('No tiene como grupo funcional una amina secundaria')

##**Indices Topologicos de la molécula**
Para los calculos de los indices, requerimos eliminar los hidrogenos para obtener el grafo molecular con los hidrógenos suprimidos, que es el que nos da los valores reales que necesitamos en cada caso.

### *Grafo molecular con hidrógenos suprimidos*

In [ ]:
new_vertices = []
for i in vertices:
  new_vertices.append(i)
new_matriz = np.array(matriz)
i = 0

while i < len(new_vertices): #Mientras sea posible acceder a los indices de la lista de vértices sin el hidrógeno
  if H_completo(new_matriz, i, new_vertices[i]) == 1: #Si seleccionamos un hidrógeno
    #Lo removemos tanto de la lista como de la matriz
    new_vertices.remove('H')
    new_matriz = np.delete(new_matriz, i, axis=0)
    new_matriz = np.delete(new_matriz, i, axis=1)
  else:
    i += 1

new_grafo = ig.Graph.Adjacency(new_matriz, mode="undirected") #Creamos un grafo con esta nueva matriz

###*Indice de Randic*


In [ ]:
def calcular_indice_randic(grafo):
    n = grafo.vcount()
    suma_indice = 0

    for u, v in itertools.combinations(grafo.vs, 2):
        d_u = grafo.degree(u)
        d_v = grafo.degree(v)
        suma_indice += 1 / (d_u * d_v)**0.5

    indice_randic = suma_indice

    return indice_randic

In [ ]:
# Calcular el índice de Randic
indice_randic = calcular_indice_randic(new_grafo)
print(f"El índice de Randic del grafo es: {indice_randic}")

###*Indice de Wiener*

In [ ]:
def calcular_indice_wiener(grafo):
    n = grafo.vcount()
    indice_wiener = 0

    for u, v in itertools.combinations(grafo.vs, 2):
        distancia_uv = grafo.distances(source=u.index, target=v.index, mode=ig.ALL)[0][0]
        indice_wiener += distancia_uv

    return indice_wiener

In [ ]:
# Calcular el índice de Wiener
indice_wiener = calcular_indice_wiener(new_grafo)
print(f"El índice de Wiener del grafo es: {indice_wiener}")

### *Caminos de wiener*

In [ ]:
def caminos_de_wiener(matriz, vertices, grafo):
  caminos = {'p1':0, 'p2':0, 'p3':0, 'p4':0, 'p5':0, 'p6':0, 'p7':0, 'p8':0} #Números de caminos de wiener necesarios para el modelo

  i = -1
  for v in grafo.vs: #Nombramos los vértices con números para que coincidan con los nombres de las aristas
    i += 1
    v['name'] = i

  matriz_resultado = crear_matriz(vertices)
  for i in range(len(vertices)):
    for j in range(len(vertices)):
      if i == j:
        matriz_resultado[i][j] = 0 #No contamos distancias de 0
      else:
        camino_mas_corto = grafo.get_shortest_paths(i, to=j, output="vpath")
        matriz_resultado[i][j] = len(camino_mas_corto[0]) - 1 #A el resultado, que resulta ser la lista de vértices recorridos, lo convertimos en su longitud y le restamos uno, que resulta en la longitud del camino
  grafo.vs['name'] = vertices #Le devolvemos su nombre original

  #Tomamos Solo una de las partes de la matriz simétrica, para no repetir números en la cuenta
  matriz_np = np.array(matriz_resultado)
  parte_superior = np.triu(matriz_np, k=1)

  #Por cada longitud i de camino encontrada, le sumamos 1 a su contador pi
  for i in range(len(vertices)):
    for j in range(len(vertices)):
      if parte_superior[i][j] == 1:
        caminos['p1'] += 1
      elif parte_superior[i][j] == 2:
        caminos['p2'] += 1
      elif parte_superior[i][j] == 3:
        caminos['p3'] += 1
      elif parte_superior[i][j] == 4:
        caminos['p4'] += 1
      elif parte_superior[i][j] == 5:
        caminos['p5'] += 1
      elif parte_superior[i][j] == 6:
        caminos['p6'] += 1
      elif parte_superior[i][j] == 7:
        caminos['p7'] += 1
      elif parte_superior[i][j] == 8:
        caminos['p8'] += 1
      else:
        continue

  return caminos

In [ ]:
caminos_wiener = caminos_de_wiener(new_matriz, new_vertices, new_grafo)
caminos = ", ".join([f"{llave}: {valor}" for llave, valor in caminos_wiener.items()])
print(f'Los caminos de wiener son: {caminos}')

### *Metiles*
Se hace la aclaración de que estos metiles no son los mismos que los definidos por la IUPAC (número de ramificaciones CH3).

In [ ]:
def metiles(grafo, vertices):
  grados_vertices = grafo.degree() #Obtenemos el grado de todos los vértices
  m = 0 #Número de metiles
  for i in range(len(grados_vertices)):
    if grados_vertices[i] == 1: #Si encuentra un vértice de grado 1
      for j in range(len(grados_vertices)):
        if grados_vertices[j] >= 3: #Si encuentra un vértice de grado mayor o igual a 3
          if grafo.are_connected(i, j): #Y si esos dos vértices están conectados
            m += 1 #Encontramos un metil
  return m

In [ ]:
num_metiles = metiles(new_grafo, new_vertices)
print(f'el número de metiles es: {num_metiles}')

### *Modelo para hallar el punto de ebullición de un alcano*
Se usa el modelo propuesto en el informe. Cabe recalcar que solo funciona para alcanos, y que tiene una desviación estandar de 4.3 °C

In [ ]:
def punto_de_ebullicion(new_matriz, new_vertices, caminos, metil):
  for i in range(len(new_vertices)):
    if new_vertices[i] != 'C': #Si encuentra algo que no sea un carbono
      return 'No es posible calcular la temperatura dado que no es un alcano'
    for j in range(len(new_vertices)):
      if new_matriz[i][j] == 2 or new_matriz[i][j] == 3: #Si tiene enlaces dobles o triples
        return 'No es posible calcular la temperatura dado que no es un alcano'

  modelo = -167.49997 + 84.28344*caminos['p1']**0.46072 + 15.94534*caminos['p2']**0.00348 + 17.42198*caminos['p3']**0.53517 + 11.16515*caminos['p4']**0.00164 + 4.74582*caminos['p5']**0.00089 + 5.23270*caminos['p6']**0.00143 + 6.67018*caminos['p7']**0.14687 + 5.27829*caminos['p8']**0.96677 - 5.53723*metil**0.00072
  return modelo

In [ ]:
ebullicion = punto_de_ebullicion(new_matriz, new_vertices, caminos_wiener, num_metiles)
if(type(ebullicion) == float):
  print(f'Su punto de ebullición aproximado es: {ebullicion}')
else:
  print(ebullicion)

##**Resumen**

In [ ]:
print(f"Resumen de la molecula {formula}: ")
print("")
visualizar_grafo(matriz, formula, vertices)
print("")

print(f'''Masa total: {masa_total_resultante} U
La masa máxima se concentra en: {grafo.vs[vertice_max_suma_masa]['name']}
Electronegatividad total: {electronegatividad_total_resultante} kJ/mol
La electronegatividad máxima se concentra en: {grafo.vs[vertice_max_suma_carga]['name']}
Su índice de Randic es: {indice_randic}
Su índice de Wiener es: {indice_wiener}
Sus caminos de Wiener son:
{caminos}
Su número de metiles es: {num_metiles}
''')
print('Su grupo funcional principal es:')
if(es_acido_carboxilico(matriz,vertices)):
  print("ácido carboxilico")
elif(es_amida(matriz,vertices)):
  print("Amida")
elif(es_amina_primaria(matriz,vertices)):
  print("Amina primaria")
elif(es_amina_secundaria(matriz,vertices)):
  print("Amina secundaria")
elif(es_carbonilo(matriz,vertices)):
  print("Carbonilo")
elif(es_acido_carboxilico(matriz,vertices)):
  print("Acido carboxílico")
elif(es_ester(matriz,vertices)):
  print("Ester")
elif(es_alcano(matriz,vertices)):
  print("Alcano")
elif(es_alqueno(matriz,vertices)):
  print("Alqueno")
elif(es_alquino(matriz,vertices)):
  print("Alquino")
else:
  print("No se encuentra dentro de los grupos funcionales reconocibles.")

if(type(ebullicion) == float):
  print(f'Su punto de ebullición aproximado es: {ebullicion}°C')
